In [92]:
import numpy as np
from numpy import asarray as npa
import matplotlib
%matplotlib inline

import keras
from keras.layers import Dense, Activation, Convolution2D, Reshape, Flatten, ZeroPadding2D
from keras.layers import MaxPooling2D, UpSampling2D, Input, Dropout
from keras.models import Sequential, Model
from keras.utils import np_utils
import re

In [60]:
f = open('data.txt')
data = f.read()
f.close()

In [61]:
data = data.lower().decode('utf-8')
d = re.sub('[^A-Za-z]',' ', data)
ds = d.split()

In [76]:
ds = ds[:1000]
ds = npa(ds)
print ds[:10]

[u'the' u'project' u'gutenberg' u'ebook' u'of' u'human' u'all' u'too'
 u'human' u'by']


In [78]:
bow = {}
revbow = {}

for ix in range(ds.shape[0]):
    bow[ds[ix]] = ix
    revbow[ix] = [ds[ix]]

In [86]:
#Conversion to one hot vectors
indata = np.zeros((ds.shape[0], ds.shape[0]))  #First element is redundant
outdata = np.zeros((ds.shape[0], ds.shape[0]))  

outdata[0,0] = 1

for ix in range(1, ds.shape[0]-1):
    indata[ix,ix-1] = 0.7
    indata[ix,ix] = 1
    outdata[ix] = ix+1

In [105]:
inp = Input(shape=(ds.shape[0],))
embeddingShape = ds.shape[0]

#---------Model----------#
den1 = Dense(200, activation='sigmoid')(inp)
den2 = Dense(1000, activation='softmax')(den1)

#---------NN----------#
model = Model(input=inp, output=den2)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [190]:
model.fit(indata, outdata,
          batch_size=100,
          nb_epoch=100)

Epoch 1/100
1000/1000 [==============================] - 0s - loss: 3450425.5250 - acc: 0.0020         
Epoch 2/100
1000/1000 [==============================] - 0s - loss: 3450424.8000 - acc: 1.0000e-03     

In [193]:
def getOneHot(w1, w2):
    ix1 = bow[w1]
    ix2 = bow[w2]
    oneHot = np.zeros((2,ds.shape[0]))
    oneHot[:,ix1] = 0.7
    oneHot[:,ix2] = 1
    return oneHot

def getWordsFromOneHot(oneHot):
    idxs = ans.argsort()[-1:][::-1][:,:10]
    for ix in range(idxs.shape[1]):
        print oneHot[0,idxs[0,ix]], revbow[ix]

In [201]:
print ds[1:20]
ans = model.predict(getOneHot('the','for'))
print(getWordsFromOneHot(ans))

[u'project' u'gutenberg' u'ebook' u'of' u'human' u'all' u'too' u'human'
 u'by' u'friedrich' u'nietzsche' u'this' u'ebook' u'is' u'for' u'the'
 u'use' u'of' u'anyone']
0.000994566 [u'the']
0.000994606 [u'project']
0.000994652 [u'gutenberg']
0.000994812 [u'ebook']
0.000994917 [u'of']
0.000994922 [u'human']
0.000995173 [u'all']
0.000995322 [u'too']
0.000995331 [u'human']
0.000995431 [u'by']
None
